In [3]:
import Youtube_crawling_func as YC
import pandas as pd
from tqdm import tqdm
import datetime

#read API keys
keys = pd.read_csv("./API_KEY.txt")
keys = list(keys)

#Create generator
def generator(lst):
    #yield lst[0]
    yield lst[1]
    yield lst[2]
    # yield lst[3]
    # yield lst[4]
    # yield lst[5]
    # yield lst[6]
    # yield lst[7]
    # yield lst[8]
    # yield lst[9]
    yield "DONE"

key_gen = generator(keys)
key = next(key_gen)
print(f"Start wiht {key}")

total = pd.DataFrame()

#시작할 날짜를 형식에 맞게 적어주기
st = '2022-08-30T00:00:00Z'
filename_st = st

for d in tqdm(range(3)):
    #날짜 구간 하루로 설정
    print(st)
    dt_st = datetime.datetime.strptime(st, '%Y-%m-%dT%H:%M:%SZ')
    dt_end = dt_st + datetime.timedelta(days =1)
    end = datetime.datetime.strftime(dt_end, '%Y-%m-%dT%H:%M:%SZ')
    options = YC.EasyDict({'channelId':'UCcQTRi69dsVYHN3exePtZ1A', "order": 'date', "publishedAfter": st, "publishedBefore": end})

    # API key 유효성 확인
    try:
        df, v_id_list = YC.video_info(options, key)
        cdf = YC.comment(v_id_list, key)
        capdf = YC.captions(v_id_list , df['cap'].to_list())
    
    # 코인 다썼으면 다음 API key로 변경
    except:
        print(f"API key change  {key} --->")
        key = next(key_gen)
        print(f"--------------->{key}")

        # 마지막 key까지 다썼으면 웹스크래핑 종료
        if key == "DONE":
            break

        # 아직 key가 남아있으면 다음것으로 변경
        else:
            print('try with new key')
            df, v_id_list = YC.video_info(options, key)
            cdf = YC.comment(v_id_list, key)
            capdf = YC.captions(v_id_list , df['cap'].to_list())

    # 모아둔 데이터에 하루치 데이터 merge
    finally:
        day = pd.merge(df,cdf, how='left', on='video_id')
        day = pd.merge(day,capdf, how='left', on='video_id')
        total = pd.concat([total, day], axis = 0)
        print(f"day : {day.shape[0]} // total : {total.shape[0]}")
        st = end


# 저장
total = total.reset_index(drop = True)
total.to_pickle(f"../data/KBS_{filename_st[:10]}_{end[:10]}.pkl")

Start wiht 이렇게 콤마로 구분해서 띄어쓰기 없이


  0%|          | 0/3 [00:00<?, ?it/s]

2022-08-30T00:00:00Z
API key change  이렇게 콤마로 구분해서 띄어쓰기 없이 --->
--------------->한줄로 나열해서 저장해주세요
try with new key


  0%|          | 0/3 [00:00<?, ?it/s]


NameError: name 'df' is not defined